# Lab 4.1 - OpenCompass 評估實戰
## Notebook 01: 環境配置與基準準備

**學習目標**:
1. 安裝並配置 OpenCompass 評估框架
2. 準備評估數據集 (C-Eval, CMMLU 子集)
3. 加載待評估模型 (Llama-2-7B, Qwen-7B)
4. 驗證環境與依賴

**預計時間**: 30-45 分鐘

---

## 1. 環境檢查

首先驗證 GPU 和基本依賴是否正常。

In [ ]:
import torch
import sys
from pathlib import Path

print("=" * 60)
print("環境檢查")
print("=" * 60)

# Python 版本
print(f"Python 版本: {sys.version.split()[0]}")

# PyTorch 版本
print(f"PyTorch 版本: {torch.__version__}")

# CUDA 可用性
print(f"CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA 版本: {torch.version.cuda}")
    print(f"GPU 數量: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"    記憶體: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ 警告: CUDA 不可用,將使用 CPU 運行 (速度會非常慢)")

print("=" * 60)

## 2. 安裝 OpenCompass

OpenCompass 是一個全面的 LLM 評估平台,支持多種評估基準。

In [ ]:
%%bash

# 檢查是否已安裝 OpenCompass
if python -c "import opencompass" 2>/dev/null; then
    echo "✅ OpenCompass 已安裝"
    python -c "import opencompass; print(f'版本: {opencompass.__version__}')"
else
    echo "📦 安裝 OpenCompass..."
    
    # Clone OpenCompass repository
    if [ ! -d "opencompass" ]; then
        git clone https://github.com/open-compass/opencompass.git
    fi
    
    cd opencompass
    pip install -e . --quiet
    
    echo "✅ OpenCompass 安裝完成"
fi

## 3. 準備評估數據集

我們將使用以下基準的子集進行評估:
- **C-Eval**: 中文評估基準 (52 個學科)
- **CMMLU**: 中文多任務語言理解 (67 個任務)

為了加快實驗速度,我們將使用部分學科的子集。

In [ ]:
from datasets import load_dataset
import os

# 設定數據目錄
DATA_DIR = Path("./data")
DATA_DIR.mkdir(exist_ok=True)

print("📥 下載評估數據集...")
print("=" * 60)

# C-Eval 數據集
try:
    print("\n1. C-Eval 數據集")
    ceval_subjects = [
        "computer_science",      # STEM
        "physics",               # STEM
        "mathematics",           # STEM
        "chinese_language_and_literature",  # Humanities
        "history",               # Humanities
        "law",                   # Social Science
        "economics"              # Social Science
    ]
    
    ceval_data = {}
    for subject in ceval_subjects:
        dataset = load_dataset(
            "ceval/ceval-exam",
            subject,
            split="val",
            trust_remote_code=True
        )
        ceval_data[subject] = dataset
        print(f"  ✅ {subject}: {len(dataset)} 題")
    
    print(f"\n  總計: {sum(len(d) for d in ceval_data.values())} 題")
    
except Exception as e:
    print(f"  ❌ 錯誤: {e}")
    print("  💡 提示: 可能需要 HuggingFace 帳號認證")
    ceval_data = {}

print("\n" + "=" * 60)
print("✅ 數據集準備完成")

### 檢查數據集格式

讓我們查看一個樣本,了解數據格式。

In [ ]:
if ceval_data:
    # 取得第一個學科的第一題
    first_subject = list(ceval_data.keys())[0]
    sample = ceval_data[first_subject][0]
    
    print(f"學科: {first_subject}")
    print("=" * 60)
    print(f"問題: {sample['question']}")
    print(f"\n選項:")
    print(f"  A: {sample['A']}")
    print(f"  B: {sample['B']}")
    print(f"  C: {sample['C']}")
    print(f"  D: {sample['D']}")
    print(f"\n正確答案: {sample['answer']}")
    print("=" * 60)
else:
    print("⚠️ C-Eval 數據未加載,跳過樣本顯示")

## 4. 準備評估模型

我們將評估兩個 7B 參數的模型:
- **Llama-2-7B**: Meta 的開源基礎模型
- **Qwen-7B**: 阿里巴巴的中文優化模型

為了加速載入和降低記憶體需求,我們將使用 4-bit 量化版本。

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
import torch

# 量化配置 (4-bit)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

def load_model_and_tokenizer(model_name: str, use_quantization: bool = True):
    """
    加載模型和 tokenizer
    
    Args:
        model_name: 模型名稱或路徑
        use_quantization: 是否使用量化
    
    Returns:
        model, tokenizer
    """
    print(f"\n📥 加載模型: {model_name}")
    print("=" * 60)
    
    # 加載 tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True
    )
    
    # 加載模型
    model_kwargs = {
        "trust_remote_code": True,
        "device_map": "auto",
    }
    
    if use_quantization and torch.cuda.is_available():
        model_kwargs["quantization_config"] = quantization_config
        print("  使用 4-bit 量化")
    
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        **model_kwargs
    )
    
    # 顯示模型資訊
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"  ✅ 模型加載成功")
    print(f"  總參數量: {total_params:,}")
    print(f"  可訓練參數: {trainable_params:,}")
    
    if torch.cuda.is_available():
        memory_allocated = torch.cuda.memory_allocated() / 1024**3
        print(f"  GPU 記憶體使用: {memory_allocated:.2f} GB")
    
    print("=" * 60)
    
    return model, tokenizer

### 加載 Llama-2-7B

In [ ]:
# 注意: 需要先從 HuggingFace 申請 Llama-2 的訪問權限
# 如果沒有權限,可以使用其他開源模型如 Mistral-7B

LLAMA_MODEL_NAME = "meta-llama/Llama-2-7b-hf"

try:
    llama_model, llama_tokenizer = load_model_and_tokenizer(
        LLAMA_MODEL_NAME,
        use_quantization=True
    )
except Exception as e:
    print(f"❌ 錯誤: {e}")
    print("\n💡 提示: 如果沒有 Llama-2 訪問權限,請:")
    print("  1. 訪問 https://huggingface.co/meta-llama/Llama-2-7b-hf")
    print("  2. 申請訪問權限")
    print("  3. 使用 huggingface-cli login 登入")
    llama_model = None
    llama_tokenizer = None

### 加載 Qwen-7B

In [ ]:
QWEN_MODEL_NAME = "Qwen/Qwen-7B"

try:
    qwen_model, qwen_tokenizer = load_model_and_tokenizer(
        QWEN_MODEL_NAME,
        use_quantization=True
    )
except Exception as e:
    print(f"❌ 錯誤: {e}")
    print("\n💡 提示: 請確認網路連接正常")
    qwen_model = None
    qwen_tokenizer = None

## 5. 測試模型推理

在正式評估前,讓我們測試模型是否能正常生成回答。

In [ ]:
def test_model_inference(model, tokenizer, prompt: str, max_length: int = 100):
    """
    測試模型推理功能
    
    Args:
        model: 模型
        tokenizer: Tokenizer
        prompt: 輸入提示
        max_length: 最大生成長度
    """
    if model is None or tokenizer is None:
        print("⚠️ 模型未加載,跳過測試")
        return
    
    print(f"提示: {prompt}")
    print("=" * 60)
    
    # 編碼輸入
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # 生成回答
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 解碼輸出
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print(f"回答: {response}")
    print("=" * 60 + "\n")

In [ ]:
# 測試提示
test_prompt = "什麼是機器學習?"

print("\n🧪 測試 Llama-2-7B")
test_model_inference(llama_model, llama_tokenizer, test_prompt)

print("\n🧪 測試 Qwen-7B")
test_model_inference(qwen_model, qwen_tokenizer, test_prompt)

## 6. 準備評估配置

建立評估所需的配置檔案。

In [ ]:
import json

# 評估配置
eval_config = {
    "models": {
        "llama-2-7b": {
            "name": LLAMA_MODEL_NAME,
            "loaded": llama_model is not None
        },
        "qwen-7b": {
            "name": QWEN_MODEL_NAME,
            "loaded": qwen_model is not None
        }
    },
    "datasets": {
        "ceval": {
            "subjects": list(ceval_data.keys()),
            "total_samples": sum(len(d) for d in ceval_data.values())
        }
    },
    "eval_params": {
        "batch_size": 8,
        "max_length": 512,
        "temperature": 0.0,
        "do_sample": False
    }
}

# 保存配置
config_path = DATA_DIR / "eval_config.json"
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(eval_config, f, indent=2, ensure_ascii=False)

print("✅ 評估配置已保存")
print(f"路徑: {config_path}")
print("\n配置內容:")
print(json.dumps(eval_config, indent=2, ensure_ascii=False))

## 7. 環境驗證總結

In [ ]:
print("\n" + "=" * 60)
print("環境驗證總結")
print("=" * 60)

checks = [
    ("CUDA 可用", torch.cuda.is_available()),
    ("C-Eval 數據加載", len(ceval_data) > 0),
    ("Llama-2-7B 加載", llama_model is not None),
    ("Qwen-7B 加載", qwen_model is not None),
    ("評估配置創建", config_path.exists())
]

for check_name, status in checks:
    status_icon = "✅" if status else "❌"
    print(f"{status_icon} {check_name}")

all_passed = all(status for _, status in checks[1:])  # 跳過 CUDA (可選)

print("\n" + "=" * 60)
if all_passed:
    print("✅ 所有檢查通過! 可以繼續進行評估 (Notebook 02)")
else:
    print("⚠️ 部分檢查未通過,請解決上述問題後再繼續")
print("=" * 60)

## 📝 總結

在本 notebook 中,我們完成了:

1. ✅ 環境檢查 (CUDA, PyTorch)
2. ✅ 安裝 OpenCompass 評估框架
3. ✅ 準備 C-Eval 評估數據集
4. ✅ 加載 Llama-2-7B 和 Qwen-7B 模型 (4-bit 量化)
5. ✅ 測試模型推理功能
6. ✅ 創建評估配置文件

### 下一步

前往 **02-Evaluate.ipynb** 執行實際評估流程。

---

**記得**:
- 如果遇到 OOM (Out of Memory) 錯誤,可以減小 batch_size
- C-Eval 完整評估需要 2-3 小時,建議先用子集測試
- 保存重要的中間結果,避免需要重新運行
